In [ ]:
from cs103 import *
from typing import NamedTuple, List
import csv
import cx_Oracle

# Philosophy
Let's put it all together!

This assignment is meant to give you some experience with using code to work with a database. It is also an opportunity to use all of the knowledge we have learned so far to see how the various topics from our class fit together.

# Step 1: Create an ER diagram

Pick a non-blacklisted topic (see below) and draw an ER diagram for that topic. The ER diagram does not have to fully encapsulate all aspects of the topic (e.g., if you were doing an ER diagram for a hospital, you can focus on one aspect like patient management instead of the full hospital).

Your ER diagram must include:
* At least 5 different entities and 5 relationships
    * At least one relationship is required to have a key constraint. The relationship with the key constraint cannot be a weak entity.
    * At least one relationship is required to have a participation constraint. The relationship with the participation constraint cannot be a weak entity.
    * To keep things interesting (and to get some more practice with relational tables), We warmly recommend having a mix of different relationship types.
* All entities must have their primary key clearly identified using the notation we learned in class.
* At least one meaningful ISA OR weak entity relationship. You can have both an ISA and a weak entity relationship if you wish.
    * An ISA relationship only counts as one entity. E.g., a parent entity with six children counts as one entity, not seven.
    * A week entity does not count as part of the "5 different entities" requirement

The ER diagram must use the notation discussed in class. ER diagrams that use alternate notation formats may be penalized.

**Topic Blacklist**: Some project topics that you may not use are:
* Employees and departments (projects, managers, employees) –textbook example
* A bookstore or library (used in CPSC 304)
* Car rental service (used in CPSC 304)
* Common website/textbook examples such as:
    * Banking (accounts, customers)
    Airlines (flights, planes, passengers, destinations)
    * Retail sales (customers, products, orders, line orders) – another common example.
        * If you want to do something in retail, consider other aspects of retail, like maybe inventory tracking, shipping, etc., and don’t focus on ordering.
    * Hospital or medical clinic
* Any other examples presented in the course including but not limited to schools/courses/departments, movie/moviestars, and a grocery store
* Any project that you have previously done (i.e., you must do a new project and cannot reuse an existing project idea)

# Step 1a: Describe your ER diagram
Provide a brief description to give some context to your ER diagram. What is it trying to model and what is the purpose of designing a database for this particular situation?

You should include any assumptions that you have based on your ER diagram on. 

If you have any other pieces of information that you feel the grader should be aware of, you should also include it here. In order words, when a TA evaluates your ER diagram for correctness and reasonableness, what should they know?

# Step 1b: Insert the ER diagram into the notebook
1. You will need to first export your ER diagram as a JPG file.
1. Upload your JPG file to the same folder as this notebook (you can drag/drop in Juypter to upload something).
1. Click into the cell below. Then, go to Edit -> Insert Image and choose your ER diagram.
1. You should see some text (similar to the example below) appear in the cell.
```
![name_of_file.jpg](attachment:name_of_file.jpg)
```
1. Run the cell and the image should appear.

**WARNING**: Double check your submission on Canvas to ensure the image appears. If the image does not appear, you can submit your image file along with this notebook when you run the submission cell below (make sure to hold cmd/ctrl before clicking on multiple files).

# Step 2: Write `CREATE TABLE` statements

Write the DDL statements required for the ER diagram you created in step 1. Put the **`CREATE TABLE`** statements in a `.sql` file. This file will be similar to the `bookbiz.sql` file used in a previous assignment. If you want, you can also use your solution to assignment 5 to create the `INSERT` statements for your tables.

# Step 3: Write `INSERT` statements

Each table from step 2 should have at least 8 tuples. Include the `INSERT` statements in the .sql file (similar to the `bookbiz.sql` file).

Provide a table or an image that depicts what data has been inserted into which table. An example table can be found in the cell below. You can also create tables in a spreadsheet document (or another application), take a screenshot, and insert it into the cell below (Edit -> Insert Image).

Depict your relational instance in this cell. You can replace the example Student table we have below.

**Student Table**:

| Student ID | Name | Address | Phone | Major | Home Inst | Start Date |
|-|-|-|-|-|-|-|
|12345676|Irena Zhang|234 Beach Ave. Vancouver, BC |6041234567|Engineering|-|-|

# Step 4: Transfer the `.sql` file from step 4 to the UBC CS server and run it

Follow the same steps as what had to be done for `bookbiz.sql` in a previous assignment.

You can also write a function to read your `.sql` file and execute each command in the database.

**Note**: When you are finished the assignment, please don't delete the tables from your database. Your TAs may wish to do some testing and it would help them immensely if the tables are present.

# Step 5: To tunnel or not to tunnel. That is the question.
### The answer is yes. You need to tunnel.

The Oracle database is hosted on a protected department machine. This means that depending on where you are working (i.e., [Syzygy](https://www.students.cs.ubc.ca/~cs-368/resources/connecting-to-db.html#syzygy) vs. a [local installation of Jupyter on your own machine](https://www.students.cs.ubc.ca/~cs-368/resources/connecting-to-db.html#local)), you will have to do something different to ensure that you can successfully connect to the database.

You cannot skip this step. You will not be able to connect to the database if you do.

In [ ]:
# We've decided to use the singleton pattern here to ensure that we are always using the same
# connection to do something in the database
# The CS department's Oracle database does not support more than two connections at a time
# and generally having multiple connections may result in unexpected behaviour (particularly if
# you have multiple threads as one connection might change something unbeknownst to the other connection(s))

def get_connection(username, password):
    """
    Returns a connection to the database to enable queries/updates.
    """
    global connection
    
    if connection:
        dsn = cx_Oracle.makedsn("localhost", 1522, service_name="stu")
        connection = cx_Oracle.connect(user=username, password=password, dsn=dsn, encoding="UTF-8")
    
    return connection
    
def close_connection():
    """
    Closes the connection to the database.
    """
    connection.close()

# Step 6: Query the database

Now that we can connect to the database, it's time to start writing some code so that we can execute some queries.

If you are running into trouble, try these two resources:
* https://cx-oracle.readthedocs.io/en/latest/user_guide/connection_handling.html
* https://cx-oracle.readthedocs.io/en/latest/user_guide/sql_execution.html

# Step 7: Update data in a table

In a previous assignment, we briefly mentioned the issue of SQL injection (the [XKCD comic about Bobby Tables](https://xkcd.com/327/)). SQL injection is when someone uses an opportunity for user input to run their own query. It is considered very dangerous to directly concatenate whatever a user gives you.

To get around the issue of directly executing anything that a user provides, we use [bind variables](https://cx-oracle.readthedocs.io/en/latest/user_guide/bind.html).

Write a function that accepts some user input that dictates what the value(s) in your tuple(s) will be. E.g., In our ongoing UoU example from other assignments, this might be a function that we use to update someone's final course grade when we realize that we inputted the wrong grade.

If updating data does not make sense given the domain/scenario that you are working with, you can modify this question to write a function that will insert new tuples into a table. For the UoU example, this might be a function that inserts tuples into the Enrollment table because a student has signed up for a new set of courses.

No matter which version you use, your function should make use of bind variables (rather than directly concatenating the values into an `UPDATE`/`INSERT` statement.

In [ ]:
# Write your solution to step 7 here


# Step 8: Fetch data from the database
To make things more interesting for you, we would like for you to have the freedom to create your own query based on your database design from step 1.

Note we are looking for a **meaningful** query. By meaningful, we mean that the query should make sense and be somewhat reasonable (instead of randomly trying to mash relations together).

Here are some requirements your query has to meet:
* It has to use multiple tables (whether these are multiple tables listed in the `FROM` clause or multiple tables due to the use of nested queries is up to you).
* It has to use a `HAVING` clause.
* It has to use an aggregate operator (e.g., `COUNT`, `MIN`,`SUM`).
* It has to return at least three tuples in the result.
* The query has to contain all the data required (i.e., you **cannot** fetch everything and then filter the results in the client)

# Step 8a: Write an English description of the query

In the cell below, write an English description of your query. Do **not** describe how the query works. I.e., we do not want a description like "First we will find the rows with this value and then we will take the first value from this list and add it etc.".

Imagine you are trying to describe this query to a five year old child. What is the query meant to do?

<font color="blue">Write your solution to step 8a here. Please leave the font tags intact as they are meant to help your TAs easily identify your answer</font>

# Step 8b: Execute your query and fetch the result

Write a function that will execute your query and print out the results of that query. If you are stuck on how to do this, please read this [page](https://cx-oracle.readthedocs.io/en/latest/user_guide/sql_execution.html).

In [ ]:
# Write your solution to step 8b here


# Step 9: Submit your work!
You can either submit your work by downloading all of your files and uploading them to Canvas or you can run the submission cell below.

If you are running the submission cell, you will need to rename your .sql file to have a .txt extension (e.g., bookbiz.sql.txt). 

No matter which method you use to submit your work, be sure to submit:
* This Jupyter file
* The .sql file that contains your CREATE TABLE and INSERT statements
* Your ER diagram (The image should already be included in the Jupyter file but in case something goes wrong and it doesn't show up on Canvas, you can submit the image separately).

In [ ]:
# NOTE: You should not be able to edit this cell. Just run it to start the process of submiting your code.
from cs103 import submit

COURSE = 83423
ASSIGNMENT = 1136661

submit(COURSE, ASSIGNMENT)

# If your submission fails, SUBMIT ANYWAY by downloading your files and uploading them to Canvas. 
# You can learn how on the page 
# "How to submit your Jupyter notebook" on our Canvas site.